In [2]:
!pip install --upgrade torch torchvision torchaudio
!pip install pytorch_lightning
!pip install wandb
!pip install gdown

^C
   ---------------------------------------- 0.0/815.2 kB ? eta -:--:--
   ------------------------- -------------- 524.3/815.2 kB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 815.2/815.2 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/866.2 kB ? eta -:--:--
   ---------------------------------------- 866.2/866.2 kB 7.7 MB/s eta 0:00:00


   ---------------------------------------- 0.0/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/199.4 MB ? eta -:--:--
   ---------------------------------------- 0.5/199.4 MB 1.7 MB/s eta 0:01:59
   ---------------------------------------- 0.8/199.4 MB 1.6 MB/s eta 0:02:05
   ---------------------------------------- 1.3/199.4 MB 1.7 MB/s eta 0:01:59
   ---------------------------------------- 1.6/199.4 MB 1.5 MB/s eta 0:02:10
   ---------------------------------------- 1.6/199.4 MB 1.5 MB/s eta 0:02:10
   ---------------------------------------- 2.1/199.4 MB 1.4 MB/s eta 0:02:22
   ---------------------------------------- 2.1/199.4 MB 1.4 MB/s eta 0:02:22
   ---------------------------------------- 2.4/199.4 MB 1.4 MB/s eta 0:02:26
    --------------------------------------- 2.6/199.4 MB 1.2 MB/s eta 0:02:41
    --------------------------------------- 2.9/199.4 MB 1.3 MB/s eta 0:02:34
    --------------------------------------- 3.4/199.4 MB 1.3 MB/s eta 0:02:27


  You can safely remove it manually.


In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import glob
import random
import numpy as np
from torchmetrics import Accuracy, Precision, Recall
import wandb
from pytorch_lightning.loggers import WandbLogger
import torchvision.transforms as T
from custom_tf import apply_transform_list

OSError: [WinError 126] No se puede encontrar el módulo especificado. Error loading "c:\nico\wildfire2024\.venv\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [ ]:
class FireSeriesDataset(Dataset):
    def __init__(self, root_dir, img_size=112, transform=None):
        self.transform = transform
        self.sets = glob.glob(f"{root_dir}/**/*")
        self.img_size=img_size
        random.shuffle(self.sets)

    def __len__(self):
        return len(self.sets)

    def __getitem__(self, idx):
        img_folder = self.sets[idx]
        img_list = glob.glob(f"{img_folder}/*.jpg")

        labels = []
        for file in img_list:
            label_file = file.replace("images", "labels").replace(".jpg", ".txt")
            with open(label_file, "r") as f:
                lines = f.readlines()

            labels.append(np.array(lines[0].split(" ")[1:5]).astype("float"))

        labels = np.array(labels)
        xc = np.median(labels[:, 0]) 
        yc = np.median(labels[:, 1]) 
        wb = np.max(labels[:, 2]) 
        hb = np.max(labels[:, 3]) 

        # Load all images first
        images = [Image.open(file) for file in img_list]
        w, h = images[0].size  

        crop_size = max(wb*h, hb*h)
        if crop_size < self.img_size:
            crop_size = self.img_size

        x0 = int(xc * w - crop_size / 2)
        y0 = int(yc * h - crop_size / 2)
        x1 = int(xc * w  + crop_size / 2)
        y1 = int(yc * h + crop_size / 2)

        img_list = []

        for im in images:
            cropped_image = im.crop(
                (x0, y0, x1,y1))
            
            cropped_image = cropped_image.resize((self.img_size, self.img_size))
            img_list.append(cropped_image)

        tensor_list = apply_transform_list(img_list)
        

        return torch.cat(tensor_list, dim=0), int(img_folder.split("/")[-2])